In [32]:
import numpy as np
import pandas as pd
import sys, time, copy, time, random
import pickle as pkl
from sklearn import covariance
from sklearn.covariance import GraphicalLasso
import pandas as pd 
import numpy.linalg as LA
import networkx as nx
import cvxpy as cp
from numpy.linalg import norm as nm
from numpy.linalg import inv, pinv 
from scipy.stats import spearmanr
import warnings
warnings.filterwarnings('ignore')
from control import dare

In [33]:
def choose_graph(graph_mode,p):
    g = graph_mode
    if g == 'bara':
        #print(g,p)
        m = int(p)
        graph = nx.barabasi_albert_graph(n, m)
    elif g == 'er':
        p_er = p
        #print(g,p)
        graph = nx.erdos_renyi_graph(n, p_er, directed=True) 
    elif g == 'ws':
        k = int(p)
        #print(g,p)
        graph = nx.watts_strogatz_graph(n, k, p_ws)
    return graph

def choose_graph1(graph_mode):
    g = graph_mode
    if g == 'bara':
        graph = nx.barabasi_albert_graph(n, m)
    elif g == 'er':
        graph = nx.erdos_renyi_graph(n, p_er, directed=True) 
    elif g == 'ws':
        graph = nx.watts_strogatz_graph(n, k, p_ws)
    return graph

def weighted_matrix(n):
    sign = np.random.choice([-1,1],size = (n,n))
    W = np.multiply(sign,np.random.randint(low = 1, high = 5, size = (n,n)))
    return W

def action_series(noise_level, graph, flag_m, T):
    n = graph.number_of_nodes()
    A = nx.adjacency_matrix(graph).todense()
    W = weighted_matrix(n)
    G = np.multiply(W,A)
    spectral_radius = np.max([np.abs(i) for i in np.linalg.eigvals(G)])
    G = G.T/(spectral_radius+0.01)
    # generate alpha
    alpha_cov = np.identity(n)
    alpha_com = 10*np.random.multivariate_normal(mean = np.zeros(n), cov = alpha_cov)

    # generate u n*T
    u_cov = np.identity(n) # Sw
    U_com = np.random.multivariate_normal(mean = np.zeros(n), cov = u_cov, size = T).reshape(T, n)
    
    # generate x0
    x0_cov = np.identity(n) # Sw
    x0_com = 10*np.random.multivariate_normal(mean = np.zeros(n), cov = x0_cov)
    # generate x series n*(T+1)
    X_com = np.zeros((T+1,n))
    X_com[0] = x0_com
    
    noise = noise_level * np.random.multivariate_normal(mean = np.zeros(n), cov = np.identity(n), size = T).reshape(T, n)
    for i in range(T):
        X_com[i+1] = alpha_com + X_com[i]@G.T + U_com[i]*flag_m + noise[i]
    X_com = X_com.T
    U_com = U_com.T
    return X_com, U_com, alpha_com, G, u_cov

def Y_Z_V_series(X_com, U_com):
    n, m = X_com.shape
    X_com = X_com.T
    U_com = U_com.T
    Y_com = np.zeros((m-2, n))
    Z_com = np.zeros((m-2, n))
    V_com = np.zeros((m-2, n))
    #noise = noise_level * np.random.multivariate_normal(mean = np.zeros(n), cov = np.identity(n), size = m-1).reshape(m-1, n)
    for i in range(m-2):
        Y_com[i] = X_com[i+1] - X_com[i] #+ noise[i]
        Z_com[i] = X_com[i+2] - X_com[i+1] #+ noise[i+1]
        V_com[i] = U_com[i+1] - U_com[i]
    Y_com = Y_com.T
    Z_com = Z_com.T
    V_com = V_com.T
    return Y_com, Z_com, V_com

# SSSI

In [34]:
def learning_graph_1(theta, Y_com, Z_com, V_com, B, G_truth, u_cov):
    G = cp.Variable((n,n),name = 'G')
    obj = cp.Minimize(0.5*cp.norm(Z_com - G@Y_com - B*V_com,'fro')**2 + theta*cp.pnorm(G, 1))
    prob = cp.Problem(obj)
    prob.solve()
    G_learn_1 = G.value
    K, G_stable = project2stable(G_learn_1, u_cov)
    relative_error = np.linalg.norm(G_stable - G_truth,'fro')/np.linalg.norm(G_truth,'fro')
    return relative_error

def project2stable(G_learn, u_cov):
    delta = 0.00000001
    Q = np.identity(n)
    X, L, K = dare(G_learn, np.identity(n), Q, inv(2*delta*u_cov)) 
    return K, G_learn - K

# SLS

In [35]:
def least_square(Y_com, Z_com, V_com, B, G_truth, u_cov):
    G_ls = (Z_com - B*V_com)@Y_com.T @ pinv(Y_com@Y_com.T)
    K, G_stable = project2stable(G_ls, u_cov)
    relative_error = np.linalg.norm(G_stable - G_truth,'fro')/np.linalg.norm(G_truth,'fro')
    return relative_error

# SL2LS

In [36]:
def l2_norm(theta, Y_com, Z_com, V_com, B, G_truth, u_cov):
    G = cp.Variable((n,n),name = 'G')
    obj = cp.Minimize(0.5*cp.norm(Z_com - G@Y_com - B*V_com,'fro')**2 + theta*cp.norm(G, 'fro')**2)
    prob = cp.Problem(obj)
    prob.solve()
    G_l2 = G.value
    K, G_stable = project2stable(G_l2, u_cov)
    relative_error = np.linalg.norm(G_stable - G_truth,'fro')/np.linalg.norm(G_truth,'fro')
    return relative_error

In [37]:
n = 100
num_simulations = 20

# Noise Level

In [38]:
T = 40
noise_levels = [0.05,0.1,0.15,0.2,0.25,0.3]
flag_m = np.random.choice([0,1],size = n)
B = flag_m.reshape(n,1) @ np.ones((1,T-1))
graph_modes = ['er','ws','bara']

#0.0979591836734694
# theta_l2_norm = {'er': 1.495,'ws':0.759,'bara':1.806}
# theta_learn1 = {'er': 0.0118,'ws':0.0143,'bara':0.0133}
#0.00125

theta_l2_norm = {'er': 0.098,'ws':0.098,'bara':0.098}
theta_learn1 = {'er': 0.00125,'ws':0.00125,'bara':0.00125}

#ER 
p_er = 0.1

# WS
p_ws = 0.2
k = 5

# BA
m = 2

# df_graph = []
# df_error = []
# df_method = []
# df_noise = []
# parameter = {'er': 'p', 'ws':'k', 'bara':'m'}
# p_value = {'er': p_er, 'ws': k, 'bara': m}



for graph_mode in graph_modes:
    df_graph = []
    df_error = []
    df_method = []
    df_noise = []
    for noise_level in noise_levels:
        cnt = 0
        while cnt < num_simulations:
            graph = choose_graph1(graph_mode)
            X_com, U_com, alpha_com, G_truth, u_cov = action_series(noise_level, graph, flag_m, T)
            Y_com, Z_com, V_com = Y_Z_V_series(X_com, U_com)
            learn1_theta = theta_learn1[graph_mode]
            learn1_error = learning_graph_1(learn1_theta, Y_com, Z_com, V_com, B, G_truth, u_cov)
            df_graph.append(graph_mode)
            df_noise.append(noise_level)
            df_error.append(learn1_error)
            df_method.append('SSSI')
            print('Noise level ', noise_level,' cnt: ',cnt,' Relative Error of Algorithm 1: ',learn1_error)
            
            ls_error = least_square(Y_com, Z_com, V_com, B, G_truth, u_cov)
            df_graph.append(graph_mode)
            df_noise.append(noise_level)
            df_error.append(ls_error)
            df_method.append('SLS')
            print('Noise level ', noise_level,' cnt: ',cnt,' Relative Error of Stablized least square: ',ls_error)
            
            l2_norm_theta = theta_l2_norm[graph_mode]
            l2_norm_error = l2_norm(l2_norm_theta, Y_com, Z_com, V_com, B, G_truth, u_cov)
            df_graph.append(graph_mode)
            df_noise.append(noise_level)
            df_error.append(l2_norm_error)
            df_method.append('SL2LS')
            print('Noise level ', noise_level,' cnt: ',cnt,' Relative Error of Stablized L2 norm: ',l2_norm_error)

            cnt = cnt +1
    print(len(df_graph),len(df_noise),len(df_error),len(df_method)) 
    columns = {'graph':df_graph, 'noise level':df_noise, 'error':df_error,'method':df_method}
    df_performance = pd.DataFrame(columns)
    df_performance.to_csv('./noise_performance_{}.csv'.format(graph_mode))


Noise level  0.05  cnt:  0  Relative Error of Algorithm 1:  0.3205385375626179
Noise level  0.05  cnt:  0  Relative Error of Stablized least square:  0.7788534706781807
Noise level  0.05  cnt:  0  Relative Error of Stablized L2 norm:  0.7789621298215118
Noise level  0.05  cnt:  1  Relative Error of Algorithm 1:  0.34428712114258014
Noise level  0.05  cnt:  1  Relative Error of Stablized least square:  0.7807204681218719
Noise level  0.05  cnt:  1  Relative Error of Stablized L2 norm:  0.780720028126599
Noise level  0.05  cnt:  2  Relative Error of Algorithm 1:  0.325878546300637
Noise level  0.05  cnt:  2  Relative Error of Stablized least square:  0.7791347784975219
Noise level  0.05  cnt:  2  Relative Error of Stablized L2 norm:  0.7791365123120256
Noise level  0.05  cnt:  3  Relative Error of Algorithm 1:  0.2638301564430555
Noise level  0.05  cnt:  3  Relative Error of Stablized least square:  0.7787587821339946
Noise level  0.05  cnt:  3  Relative Error of Stablized L2 norm:  0.77

Noise level  0.1  cnt:  12  Relative Error of Stablized L2 norm:  0.7901082603705797
Noise level  0.1  cnt:  13  Relative Error of Algorithm 1:  0.3911097966065836
Noise level  0.1  cnt:  13  Relative Error of Stablized least square:  0.7824864434588272
Noise level  0.1  cnt:  13  Relative Error of Stablized L2 norm:  0.7824763827763466
Noise level  0.1  cnt:  14  Relative Error of Algorithm 1:  0.3304651387326998
Noise level  0.1  cnt:  14  Relative Error of Stablized least square:  0.7860538604909012
Noise level  0.1  cnt:  14  Relative Error of Stablized L2 norm:  0.7860393819445615
Noise level  0.1  cnt:  15  Relative Error of Algorithm 1:  0.39620064373426683
Noise level  0.1  cnt:  15  Relative Error of Stablized least square:  0.783277617165793
Noise level  0.1  cnt:  15  Relative Error of Stablized L2 norm:  0.7832743553223255
Noise level  0.1  cnt:  16  Relative Error of Algorithm 1:  0.34040748954176536
Noise level  0.1  cnt:  16  Relative Error of Stablized least square:  0.

Noise level  0.2  cnt:  5  Relative Error of Algorithm 1:  0.36741201657595685
Noise level  0.2  cnt:  5  Relative Error of Stablized least square:  0.7884771437237496
Noise level  0.2  cnt:  5  Relative Error of Stablized L2 norm:  0.7885834989708062
Noise level  0.2  cnt:  6  Relative Error of Algorithm 1:  0.40058260789899003
Noise level  0.2  cnt:  6  Relative Error of Stablized least square:  0.7837709961411485
Noise level  0.2  cnt:  6  Relative Error of Stablized L2 norm:  0.7837404528966385
Noise level  0.2  cnt:  7  Relative Error of Algorithm 1:  0.42904249568482283
Noise level  0.2  cnt:  7  Relative Error of Stablized least square:  0.7871630326192525
Noise level  0.2  cnt:  7  Relative Error of Stablized L2 norm:  0.7871385188040728
Noise level  0.2  cnt:  8  Relative Error of Algorithm 1:  0.3627549718077888
Noise level  0.2  cnt:  8  Relative Error of Stablized least square:  0.7673817919467513
Noise level  0.2  cnt:  8  Relative Error of Stablized L2 norm:  0.7673549093

Noise level  0.25  cnt:  17  Relative Error of Stablized L2 norm:  0.7942946263492751
Noise level  0.25  cnt:  18  Relative Error of Algorithm 1:  0.40752104955333524
Noise level  0.25  cnt:  18  Relative Error of Stablized least square:  0.7880209265121776
Noise level  0.25  cnt:  18  Relative Error of Stablized L2 norm:  0.7879978239301136
Noise level  0.25  cnt:  19  Relative Error of Algorithm 1:  0.4024462331017518
Noise level  0.25  cnt:  19  Relative Error of Stablized least square:  0.7831470138698307
Noise level  0.25  cnt:  19  Relative Error of Stablized L2 norm:  0.7831128544132645
Noise level  0.3  cnt:  0  Relative Error of Algorithm 1:  0.5240622786362786
Noise level  0.3  cnt:  0  Relative Error of Stablized least square:  0.7946575633419851
Noise level  0.3  cnt:  0  Relative Error of Stablized L2 norm:  0.7944204958267772
Noise level  0.3  cnt:  1  Relative Error of Algorithm 1:  0.5306709983453577
Noise level  0.3  cnt:  1  Relative Error of Stablized least square:  

Noise level  0.05  cnt:  10  Relative Error of Algorithm 1:  0.19167661147300408
Noise level  0.05  cnt:  10  Relative Error of Stablized least square:  0.7702960717967923
Noise level  0.05  cnt:  10  Relative Error of Stablized L2 norm:  0.7703074695711934
Noise level  0.05  cnt:  11  Relative Error of Algorithm 1:  0.15342811307236437
Noise level  0.05  cnt:  11  Relative Error of Stablized least square:  0.7723614452518571
Noise level  0.05  cnt:  11  Relative Error of Stablized L2 norm:  0.7723993546904965
Noise level  0.05  cnt:  12  Relative Error of Algorithm 1:  0.1840917406349288
Noise level  0.05  cnt:  12  Relative Error of Stablized least square:  0.7855081738103868
Noise level  0.05  cnt:  12  Relative Error of Stablized L2 norm:  0.7855037273961666
Noise level  0.05  cnt:  13  Relative Error of Algorithm 1:  0.14530396322239447
Noise level  0.05  cnt:  13  Relative Error of Stablized least square:  0.7718444570464039
Noise level  0.05  cnt:  13  Relative Error of Stablize

Noise level  0.15  cnt:  2  Relative Error of Stablized L2 norm:  0.7884410846503959
Noise level  0.15  cnt:  3  Relative Error of Algorithm 1:  0.3815264724598405
Noise level  0.15  cnt:  3  Relative Error of Stablized least square:  0.7880523895079385
Noise level  0.15  cnt:  3  Relative Error of Stablized L2 norm:  0.7875987898008576
Noise level  0.15  cnt:  4  Relative Error of Algorithm 1:  0.35585870595492225
Noise level  0.15  cnt:  4  Relative Error of Stablized least square:  0.7825364488936176
Noise level  0.15  cnt:  4  Relative Error of Stablized L2 norm:  0.7822570717525765
Noise level  0.15  cnt:  5  Relative Error of Algorithm 1:  0.24673784588131453
Noise level  0.15  cnt:  5  Relative Error of Stablized least square:  0.7881414158221509
Noise level  0.15  cnt:  5  Relative Error of Stablized L2 norm:  0.7880294613267648
Noise level  0.15  cnt:  6  Relative Error of Algorithm 1:  0.36341566895923877
Noise level  0.15  cnt:  6  Relative Error of Stablized least square:  

Noise level  0.2  cnt:  15  Relative Error of Algorithm 1:  0.3904920870852963
Noise level  0.2  cnt:  15  Relative Error of Stablized least square:  0.7812151161734086
Noise level  0.2  cnt:  15  Relative Error of Stablized L2 norm:  0.7807800317059982
Noise level  0.2  cnt:  16  Relative Error of Algorithm 1:  0.4026741652434595
Noise level  0.2  cnt:  16  Relative Error of Stablized least square:  0.7949980179185584
Noise level  0.2  cnt:  16  Relative Error of Stablized L2 norm:  0.7945748839798692
Noise level  0.2  cnt:  17  Relative Error of Algorithm 1:  0.41276666633600306
Noise level  0.2  cnt:  17  Relative Error of Stablized least square:  0.8042241571546301
Noise level  0.2  cnt:  17  Relative Error of Stablized L2 norm:  0.803902774424987
Noise level  0.2  cnt:  18  Relative Error of Algorithm 1:  0.3762808254080853
Noise level  0.2  cnt:  18  Relative Error of Stablized least square:  0.7805693484872028
Noise level  0.2  cnt:  18  Relative Error of Stablized L2 norm:  0.7

Noise level  0.3  cnt:  7  Relative Error of Stablized L2 norm:  0.8250379958817822
Noise level  0.3  cnt:  8  Relative Error of Algorithm 1:  0.3868340537405528
Noise level  0.3  cnt:  8  Relative Error of Stablized least square:  0.8061095882208681
Noise level  0.3  cnt:  8  Relative Error of Stablized L2 norm:  0.8058774800311632
Noise level  0.3  cnt:  9  Relative Error of Algorithm 1:  0.5166753631177089
Noise level  0.3  cnt:  9  Relative Error of Stablized least square:  0.8240514138256593
Noise level  0.3  cnt:  9  Relative Error of Stablized L2 norm:  0.823105184089944
Noise level  0.3  cnt:  10  Relative Error of Algorithm 1:  0.48259213640054266
Noise level  0.3  cnt:  10  Relative Error of Stablized least square:  0.8046429865553514
Noise level  0.3  cnt:  10  Relative Error of Stablized L2 norm:  0.8037581276941067
Noise level  0.3  cnt:  11  Relative Error of Algorithm 1:  0.6330217178841027
Noise level  0.3  cnt:  11  Relative Error of Stablized least square:  0.83642180

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

# Time Step

In [ ]:
time_steps = [15,20,25,30,35,40]
noise_level = 0.1

graph_modes = ['er','ws','bara']

#ER 
p_er = 0.1

# WS
p_ws = 0.2
k = 5

# BA
m = 2

theta_l2_norm = {'er': 0.098,'ws':0.098,'bara':0.098}
theta_learn1 = {'er': 0.00125,'ws':0.00125,'bara':0.00125}

# theta_l2_norm = {'er': 1.495,'ws':0.759,'bara':1.806}
# theta_learn1 = {'er': 0.0118,'ws':0.0143,'bara':0.0133}



graph_mode = graph_modes[1]

for graph_mode in graph_modes:
    df_graph = []
    df_error = []
    df_method = []
    df_T = []
    for T in time_steps:
        flag_m = np.random.choice([0,1],size = n)
        B = flag_m.reshape(n,1) @ np.ones((1,T-1))
        cnt = 0
        while cnt < num_simulations:
            graph = choose_graph1(graph_mode)
            X_com, U_com, alpha_com, G_truth, u_cov = action_series(noise_level, graph, flag_m, T)
            Y_com, Z_com, V_com = Y_Z_V_series(X_com, U_com)
            learn1_theta = theta_learn1[graph_mode]
            learn1_error = learning_graph_1(learn1_theta, Y_com, Z_com, V_com, B, G_truth, u_cov)
            df_graph.append(graph_mode)
            df_T.append(T)
            df_error.append(learn1_error)
            df_method.append('SSSI')
            print('T: ', T,' cnt: ',cnt,' Relative Error of Algorithm 1: ',learn1_error)
            
            ls_error = least_square(Y_com, Z_com, V_com, B, G_truth, u_cov)
            df_graph.append(graph_mode)
            df_T.append(T)
            df_error.append(ls_error)
            df_method.append('SLS')
            print('T: ', T,' cnt: ',cnt,' Relative Error of Stablized least square: ',ls_error)
            
            l2_norm_theta = theta_l2_norm[graph_mode]
            l2_norm_error = l2_norm(l2_norm_theta, Y_com, Z_com, V_com, B, G_truth, u_cov)
            df_graph.append(graph_mode)
            df_T.append(T)
            df_error.append(l2_norm_error)
            df_method.append('SL2LS')
            print('T: ', T,' cnt: ',cnt,' Relative Error of Stablized L2 norm: ',l2_norm_error)

            cnt = cnt +1
    print(len(df_graph),len(df_T),len(df_error),len(df_method)) 
    columns = {'graph':df_graph, 'T':df_T, 'error':df_error,'method':df_method}
    df_performance = pd.DataFrame(columns)
    df_performance.to_csv('./T_performance_{}.csv'.format(graph_mode))




In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 
from pandas.plotting import table  # EDIT: see deprecation warnings below
from matplotlib import cm
import matplotlib
name_conventions = {'er':'Erdos-Renyi', 'bara':'Barabasi-Albert', 'ws':'Watts-Strogatz'}
# Plot initialisation
font = {'family' : 'normal','weight' : 'bold','size'   : 16}
matplotlib.rc('font', **font)

def plotting_function(pt, g):
    plt.rcParams.update({'font.family': 'Helvetica', 'font.size': 14})  # Set the font family and size

    dd = pd.read_csv('./'+pt+'_performance_'+g+'.csv')
    x_name = dd.loc[1][1]
    
    plt.figure(figsize=(6, 5))
    x_name = dd.columns[2]
    bp = sns.boxplot(x=x_name, y='error', data=dd, hue='method', showfliers=False, showmeans=False, palette=sns.color_palette("Set2"), linewidth=1)
    plt.title(name_conventions[g], fontsize=18, pad=10)  # Adjust the pad value as needed
    plt.ylabel('Relative error', fontsize=18)
    plt.ylim(0, 1)
    if pt == 'T':
        plt.legend(loc = 3, prop={'size': 13}) # loc = 3/4
    if pt == 'noise':
        plt.legend(loc = 4, prop={'size': 13}) 
    plt.xlabel(x_name, fontsize=18)

    # Set x, y, and tick label font sizes
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # Set tick parameters for both axes
    plt.tick_params(axis='both', which='both', length=5, width=1)

    plt.tight_layout()
    plt.savefig('./'+pt + '_performance_{}.png'.format(g), dpi=150)


In [ ]:
types = ['T','noise']
for g in graph_modes:
    for pt in types:
        plotting_function(pt,g)